In [5]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-

"""
Advanced 15-Minute Opening Range Breakout (ORB) Backtester  →  CPR Edition
============================================================================

We keep the same scaffold/IO, but implement a CPR-based strategy (no indicators).

Strategy Overview (CPR)
-----------------------
We compute the **Central Pivot Range (CPR)** for each trading day from the **previous day's OHLC**:

Formulas (using previous day High=H, Low=L, Close=C):
    CP = (H + L + C) / 3
    BC = (H + L) / 2
    TC = 2*CP - BC

Trading Bias:
    - Above CPR → support / bullish bias
    - Below CPR → resistance / bearish bias
    - Inside CPR → avoid trades (choppy)

Trade Setups (low-frequency, high-quality):
A) Bodyless Doji *at the CPR edge*
   • At TC (top): look for **long** ONLY if the **next candle first breaks the Doji high**.
     If the next candle **first breaks the Doji low**, setup fails. Entry at NEXT bar OPEN.
   • At BC (bottom): look for **short** ONLY if the **next candle first breaks the Doji low**.
     If the next candle **first breaks the Doji high**, setup fails. Entry at NEXT bar OPEN.

B) Hammer at TC
   • A classic hammer (long lower wick) must form **touching TC**.
   • Go **long** ONLY if the **next candle first breaks the Hammer high**; if it first breaks
     the Hammer low, setup fails. Entry at NEXT bar OPEN.

Execution & Risk:
- One trade per day per symbol.
- Entry window controlled by `entry_start`..`entry_end`.
- Enforce square-off at `squareoff_time`.
- Fixed absolute ₹ risk/target per position (SL/TP in rupees across the whole position).
- Optional trailing SL in rupees per trade (ratchets only).
- Intraday leverage and Groww-like cost model applied.
- Saves `trades.csv` and prints summary.

NOTE: We retain function/section names from the ORB framework where possible so you can slot this in easily.
"""

import math
import sys
from dataclasses import dataclass
from typing import List, Dict, Tuple, Optional
import pandas as pd
import numpy as np

try:
    import yfinance as yf
except Exception:
    print("Please: pip install yfinance pandas numpy yfinance")
    sys.exit(1)

# =========================
# CONFIG
# =========================
@dataclass
class Config:
    tickers: List[str]
    start_date: str = "2025-10-01"
    end_date:   str = "2025-11-01"
    interval:   str = "5m"

    capital_per_stock: float = 100_000.0   # your cash per symbol
    intraday_leverage: float = 5.0         # 5× buying power (intraday)

    sl_rupees: float = 3000.0               # absolute ₹ stop per position
    tp_rupees: float = 6000.0              # absolute ₹ target per position

    # Direction toggles
    enable_longs: bool = True
    enable_shorts: bool = False

    # Trailing SL
    enable_trailing_sl: bool = True
    trail_rupees: float = 500.0

    # Trading session (IST)
    # (We keep the ORB placeholders in case you compare, but CPR does not use OR)
    or_start: str = "09:15"                # unused by CPR logic
    or_end:   str = "09:45"                # unused by CPR logic

    entry_start: str = "09:35"             # allow signals after market stabilizes
    entry_end:   str = "14:45"             # last time to open new trade
    squareoff_time: str = "14:45"          # force exit if still open

    timezone: str = "Asia/Kolkata"
    out_file: str = "trades.csv"

    # CPR touch tolerance (how close a candle must be to 'touch' TC/BC)
    touch_eps_pct: float = 0.0005          # 5 bps of price (~₹0.5 per ₹1000)
    # Doji strictness: require bodyless at 2-dec precision AND tiny vs range
    doji_max_body_pct_of_range: float = 0.10  # body <= 10% of bar range
    # Hammer rules
    hammer_min_lower_to_body_ratio: float = 2.0
    hammer_max_upper_to_body_ratio: float = 1.0  # upper shadow <= body

CFG = Config(
    tickers=['ADANIENT.NS', 'ADANIPORTS.NS', 'APOLLOHOSP.NS', 'ASIANPAINT.NS', 'AXISBANK.NS', 'BAJAJ-AUTO.NS', 'BAJFINANCE.NS', 'BAJAJFINSV.NS', 'BEL.NS', 'BHARTIARTL.NS', 'CIPLA.NS', 'COALINDIA.NS', 'DRREDDY.NS', 'DUMMYTATAM.NS', 'EICHERMOT.NS', 'ETERNAL.NS', 'GRASIM.NS', 'HCLTECH.NS', 'HDFCBANK.NS', 'HDFCLIFE.NS', 'HINDALCO.NS', 'HINDUNILVR.NS', 'ICICIBANK.NS', 'ITC.NS', 'INFY.NS', 'INDIGO.NS', 'JSWSTEEL.NS', 'JIOFIN.NS', 'KOTAKBANK.NS', 'LT.NS', 'M&M.NS', 'MARUTI.NS', 'MAXHEALTH.NS', 'NTPC.NS', 'NESTLEIND.NS', 'ONGC.NS', 'POWERGRID.NS', 'RELIANCE.NS', 'SBILIFE.NS', 'SHRIRAMFIN.NS', 'SBIN.NS', 'SUNPHARMA.NS', 'TCS.NS', 'TATACONSUM.NS', 'TMPV.NS', 'TATASTEEL.NS', 'TECHM.NS', 'TITAN.NS', 'TRENT.NS', 'ULTRACEMCO.NS', 'WIPRO.NS']

)

# =========================
# GROWW CHARGES (NSE intraday)
# =========================
def groww_intraday_charges(buy_turnover: float, sell_turnover: float) -> Dict[str, float]:
    """
    Compute charges for a single round-trip intraday equity trade (NSE).
    """
    def brokerage(turnover):
        fee = min(20.0, 0.001 * turnover)
        return max(5.0, fee)  # floor ₹5

    bro_buy  = brokerage(buy_turnover)
    bro_sell = brokerage(sell_turnover)

    exch_buy  = 0.0000297 * buy_turnover
    exch_sell = 0.0000297 * sell_turnover

    sebi_buy  = 0.000001 * buy_turnover
    sebi_sell = 0.000001 * sell_turnover

    ipft_buy  = 0.000001 * buy_turnover
    ipft_sell = 0.000001 * sell_turnover

    gst_buy  = 0.18 * (bro_buy  + exch_buy  + sebi_buy  + ipft_buy)
    gst_sell = 0.18 * (bro_sell + exch_sell + sebi_sell + ipft_sell)

    stt_sell = 0.00025 * sell_turnover
    stamp_buy = 0.00003 * buy_turnover

    total = (bro_buy + bro_sell + exch_buy + exch_sell +
             sebi_buy + sebi_sell + ipft_buy + ipft_sell +
             gst_buy + gst_sell + stt_sell + stamp_buy)

    return {
        "brokerage_buy": bro_buy, "brokerage_sell": bro_sell,
        "exchange_buy": exch_buy, "exchange_sell": exch_sell,
        "sebi_buy": sebi_buy, "sebi_sell": sebi_sell,
        "ipft_buy": ipft_buy, "ipft_sell": ipft_sell,
        "gst_buy": gst_buy, "gst_sell": gst_sell,
        "stt_sell": stt_sell, "stamp_buy": stamp_buy,
        "total_charges": total
    }

# =========================
# DATA
# =========================
def fetch_5m_dataframe(ticker: str, start: str, end: str, interval: str, tz: str) -> pd.DataFrame:
    df = yf.download(
        ticker,
        start=start,
        end=end,
        interval=interval,
        auto_adjust=False,
        prepost=False,
        progress=False,
        multi_level_index=False,
        group_by="column",
    )
    if df.empty:
        return df

    if isinstance(df.columns, pd.MultiIndex):
        df.columns = [" ".join([c for c in col if c]).strip() for col in df.columns.values]

    if df.index.tz is None:
        df = df.tz_localize("UTC").tz_convert(tz)
    else:
        df = df.tz_convert(tz)

    cols_map = {c: c.title() for c in df.columns}
    df.rename(columns=cols_map, inplace=True)
    keep = [c for c in ["Open", "High", "Low", "Close", "Volume"] if c in df.columns]
    df = df[keep]
    df["Ticker"] = ticker
    return df

def session_filter(df: pd.DataFrame, date: pd.Timestamp, tz: str) -> pd.DataFrame:
    d0 = pd.Timestamp(date)
    day_start = pd.Timestamp(d0.date().strftime("%Y-%m-%d") + " 09:15").tz_localize(tz)
    day_end   = pd.Timestamp(d0.date().strftime("%Y-%m-%d") + " 15:30").tz_localize(tz)
    return df.loc[(df.index >= day_start) & (df.index <= day_end)].copy()

def _ts_on_day(day: pd.Timestamp, hhmm: str, tz: str) -> pd.Timestamp:
    return pd.Timestamp(day.date().strftime("%Y-%m-%d") + f" {hhmm}").tz_localize(tz)

# =========================
# CPR UTILITIES
# =========================
def compute_daily_ohlc_map(df: pd.DataFrame) -> Dict[pd.Timestamp, Dict[str, float]]:
    """
    Build a map: date -> {'H':..., 'L':..., 'C':...} in local (IST) date buckets.
    """
    if df.empty:
        return {}
    g = df.groupby(df.index.date)
    daily = {}
    for d, sub in g:
        daily[pd.Timestamp(d)] = {
            "H": float(sub["High"].max()),
            "L": float(sub["Low"].min()),
            "C": float(sub["Close"].iloc[-1])
        }
    return daily

def get_cpr_from_prev_day(daily_map: Dict[pd.Timestamp, Dict[str, float]], day: pd.Timestamp) -> Tuple[float, float, float]:
    """
    Get (TC, CP, BC) for 'day' using the PREVIOUS available date in daily_map.
    Returns (np.nan, np.nan, np.nan) if not available.
    """
    if not daily_map:
        return np.nan, np.nan, np.nan
    keys = sorted(daily_map.keys())
    # find previous key < day
    prev_keys = [k for k in keys if k < pd.Timestamp(day.date())]
    if not prev_keys:
        return np.nan, np.nan, np.nan
    prev = prev_keys[-1]
    H, L, C = daily_map[prev]["H"], daily_map[prev]["L"], daily_map[prev]["C"]
    CP = (H + L + C) / 3.0
    BC = (H + L) / 2.0
    TC = 2.0 * CP - BC
    return float(TC), float(CP), float(BC)

def touches_level(row: pd.Series, level: float, eps: float) -> bool:
    # Candle touches a horizontal level within tolerance
    return (row["Low"] - eps) <= level <= (row["High"] + eps)

def is_bodyless_doji(row: pd.Series, max_pct_of_range: float) -> bool:
    high, low, op, cl = float(row["High"]), float(row["Low"]), float(row["Open"]), float(row["Close"])
    rng = max(1e-9, high - low)
    body = abs(cl - op)
    # Very strict: treat as "bodyless" if rounded to 2 decimals equal, and body tiny vs range
    bodyless_2dp = round(op, 2) == round(cl, 2)
    tiny_vs_range = (body / rng) <= max_pct_of_range
    return bodyless_2dp and tiny_vs_range

def is_hammer_at_tc(row: pd.Series, tc: float, eps: float, min_lower_to_body: float, max_upper_to_body: float) -> bool:
    high, low, op, cl = float(row["High"]), float(row["Low"]), float(row["Open"]), float(row["Close"])
    body = abs(cl - op)
    upper = max(0.0, high - max(op, cl))
    lower = max(0.0, min(op, cl) - low)
    if body <= 0:
        return False
    # Location: must touch TC
    if not touches_level(row, tc, eps):
        return False
    # Shape: long lower shadow, small/limited upper shadow
    cond1 = (lower / body) >= min_lower_to_body
    cond2 = (upper / body) <= max_upper_to_body
    # Body positioned toward the top half of the candle (close to high)
    cond3 = (max(op, cl) >= (low + (high - low) * 0.7))
    return cond1 and cond2 and cond3

# =========================
# SIGNAL SCAN (CPR rules)
# =========================
def scan_cpr_signal_and_entry(day_df: pd.DataFrame, day: pd.Timestamp,
                              tc: float, cp: float, bc: float,
                              cfg: Config) -> Tuple[Optional[str], Optional[pd.Timestamp], Optional[float]]:
    """
    Find the first valid CPR setup within the entry window and return:
        (direction, entry_ts, entry_price)
    Entry executes at NEXT bar open IF next bar confirms AND does not invalidate first.
    """
    if not np.isfinite(tc) or not np.isfinite(cp) or not np.isfinite(bc):
        return None, None, None

    start_ts = _ts_on_day(day, cfg.entry_start, cfg.timezone)
    last_allowed_signal_ts = _ts_on_day(day, cfg.entry_end, cfg.timezone)  # signal bar must be before this (needs next bar)

    # Price "inside CPR" → avoid trades
    first_row = day_df.loc[day_df.index >= start_ts].head(1)
    if not first_row.empty:
        px = float(first_row["Close"].iloc[0])
        if bc <= px <= tc:
            return None, None, None

    # Tolerance for "touch"
    eps = cfg.touch_eps_pct * float(day_df["Close"].median())

    # Iterate bars, checking for signal bar; then read the immediate next bar for trigger/invalid
    df_scan = day_df.loc[(day_df.index >= start_ts) & (day_df.index <= last_allowed_signal_ts)]
    for ts, row in df_scan.iterrows():
        # A) Bodyless Doji at TC → long
        if cfg.enable_longs and touches_level(row, tc, eps) and is_bodyless_doji(row, cfg.doji_max_body_pct_of_range):
            nxt = day_df.loc[day_df.index > ts].head(1)
            if nxt.empty:
                break
            nrow = nxt.iloc[0]
            # "Opposite first" conservative check on the next bar OHLC
            doji_low, doji_high = float(row["Low"]), float(row["High"])
            # If next bar's Low <= doji_low first => fail (bar sequencing unknown; be conservative)
            if float(nrow["Low"]) <= doji_low and float(nrow["High"]) < doji_high:
                pass  # invalid; continue scanning
            else:
                if float(nrow["High"]) >= doji_high and float(nrow["Low"]) > doji_low:
                    # Triggered → entry at next bar OPEN
                    entry_ts = nxt.index[0]
                    entry_price = float(nrow["Open"])
                    return "long", entry_ts, entry_price
                # If both sides pierced, treat as invalid for safety
        # A) Bodyless Doji at BC → short
        if cfg.enable_shorts and touches_level(row, bc, eps) and is_bodyless_doji(row, cfg.doji_max_body_pct_of_range):
            nxt = day_df.loc[day_df.index > ts].head(1)
            if nxt.empty:
                break
            nrow = nxt.iloc[0]
            doji_low, doji_high = float(row["Low"]), float(row["High"])
            if float(nrow["High"]) >= doji_high and float(nrow["Low"]) > doji_low:
                pass  # invalid for short (opposite high first)
            else:
                if float(nrow["Low"]) <= doji_low and float(nrow["High"]) < doji_high:
                    entry_ts = nxt.index[0]
                    entry_price = float(nrow["Open"])
                    return "short", entry_ts, entry_price

        # B) Hammer at TC → long
        if cfg.enable_longs and is_hammer_at_tc(row, tc, eps, cfg.hammer_min_lower_to_body_ratio, cfg.hammer_max_upper_to_body_ratio):
            nxt = day_df.loc[day_df.index > ts].head(1)
            if nxt.empty:
                break
            nrow = nxt.iloc[0]
            ham_low, ham_high = float(row["Low"]), float(row["High"])
            if float(nrow["Low"]) <= ham_low and float(nrow["High"]) < ham_high:
                pass  # invalid (low first)
            else:
                if float(nrow["High"]) >= ham_high and float(nrow["Low"]) > ham_low:
                    entry_ts = nxt.index[0]
                    entry_price = float(nrow["Open"])
                    return "long", entry_ts, entry_price

    return None, None, None

# =========================
# SIMULATION (with trailing SL)
# =========================
def simulate_trade(day_df: pd.DataFrame, direction: str, entry_ts: pd.Timestamp, entry_price: float,
                   capital: float, leverage: float, sl_rupees: float, tp_rupees: float,
                   squareoff_time: str, tz: str, ticker: str,
                   enable_trailing_sl: bool, trail_rupees: float):
    if entry_price is None or entry_ts is None or math.isnan(entry_price):
        return None

    buying_power = capital * leverage
    qty = int(buying_power // entry_price)
    if qty < 1:
        return None

    risk_per_share   = sl_rupees / qty
    target_per_share = tp_rupees / qty
    trail_per_share  = (trail_rupees if trail_rupees is not None else sl_rupees) / qty

    if direction == "long":
        static_sl = entry_price - risk_per_share
        tp = entry_price + target_per_share
        high_water = entry_price
        dyn_sl = static_sl
    else:
        static_sl = entry_price + risk_per_share
        tp = entry_price - target_per_share
        low_water = entry_price
        dyn_sl = static_sl

    exit_reason = None
    exit_ts = None
    exit_price = None

    after = day_df.loc[day_df.index >= entry_ts]
    for ts, row in after.iterrows():

        # Trailing stop update
        if enable_trailing_sl:
            if direction == "long":
                high_water = max(high_water, float(row["High"]))
                new_sl = max(static_sl, high_water - trail_per_share)
                dyn_sl = max(dyn_sl, new_sl)
            else:
                low_water = min(low_water, float(row["Low"]))
                new_sl = min(static_sl, low_water + trail_per_share)
                dyn_sl = min(dyn_sl, new_sl)

        if direction == "long":
            if row["Low"] <= dyn_sl:
                exit_reason, exit_ts, exit_price = ("SL-TRAIL" if enable_trailing_sl else "SL"), ts, float(dyn_sl)
                break
            if row["High"] >= tp:
                exit_reason, exit_ts, exit_price = "TP", ts, float(tp)
                break
        else:
            if row["High"] >= dyn_sl:
                exit_reason, exit_ts, exit_price = ("SL-TRAIL" if enable_trailing_sl else "SL"), ts, float(dyn_sl)
                break
            if row["Low"] <= tp:
                exit_reason, exit_ts, exit_price = "TP", ts, float(tp)
                break

        so = _ts_on_day(ts, squareoff_time, tz)
        if ts >= so:
            exit_reason, exit_ts, exit_price = "EOD", ts, float(row["Close"])
            break

    if exit_ts is None:
        last_ts = after.index[-1]
        exit_reason, exit_ts, exit_price = "EOD", last_ts, float(after.iloc[-1]["Close"])

    if direction == "long":
        gross = (exit_price - entry_price) * qty
        buy_turnover  = entry_price * qty
        sell_turnover = exit_price * qty
    else:
        gross = (entry_price - exit_price) * qty
        buy_turnover  = exit_price * qty
        sell_turnover = entry_price * qty

    fees = groww_intraday_charges(buy_turnover, sell_turnover)
    net = gross - fees["total_charges"]

    return {
        "date": entry_ts.date().isoformat(),
        "ticker": ticker,
        "direction": direction,
        "entry_time": entry_ts.isoformat(),
        "entry_price": round(entry_price, 2),
        "qty": qty,
        "sl_price_initial": round(static_sl, 2),
        "tp_price": round(tp, 2),
        "sl_trailing_enabled": enable_trailing_sl,
        "trail_rupees": trail_rupees,
        "exit_time": exit_ts.isoformat(),
        "exit_price": round(exit_price, 2),
        "gross_pnl": round(gross, 2),
        "charges": round(fees["total_charges"], 2),
        "net_pnl": round(net, 2),
        "exit_reason": exit_reason,
        "buy_turnover": round(buy_turnover, 2),
        "sell_turnover": round(sell_turnover, 2),
        "cash_capital": round(capital, 2),
        "leverage": leverage,
        "notional_at_entry": round(entry_price * qty, 2),
    }

# =========================
# BACKTEST
# =========================
def run_backtest(cfg: Config) -> pd.DataFrame:
    all_trades = []

    for ticker in cfg.tickers:
        print(f"Downloading {ticker} ...")
        df = fetch_5m_dataframe(ticker, cfg.start_date, cfg.end_date, cfg.interval, cfg.timezone)
        if df.empty:
            print(f"  WARN: No data for {ticker}")
            continue

        # Build daily OHLC map for CPR from prev day
        daily_map = compute_daily_ohlc_map(df)

        # Unique session dates
        dates = sorted(list({pd.Timestamp(ts).date() for ts in df.index}))

        for d in dates:
            day = pd.Timestamp(d)
            day_df = session_filter(df, day, cfg.timezone)
            if len(day_df) < 10:
                continue

            # CPR from PREVIOUS day
            tc, cp, bc = get_cpr_from_prev_day(daily_map, day)
            if not (np.isfinite(tc) and np.isfinite(cp) and np.isfinite(bc)):
                continue

            # Find first valid CPR signal in window
            direction, en_ts, en_price = scan_cpr_signal_and_entry(day_df, day, tc, cp, bc, cfg)
            if direction is None or en_ts is None or en_price is None:
                continue

            # Enforce latest entry time
            last_entry_ts = _ts_on_day(day, cfg.entry_end, cfg.timezone)
            if en_ts > last_entry_ts:
                continue

            trade = simulate_trade(
                day_df=day_df, direction=direction,
                entry_ts=en_ts, entry_price=en_price,
                capital=cfg.capital_per_stock, leverage=cfg.intraday_leverage,
                sl_rupees=cfg.sl_rupees, tp_rupees=cfg.tp_rupees,
                squareoff_time=cfg.squareoff_time, tz=cfg.timezone,
                ticker=ticker,
                enable_trailing_sl=cfg.enable_trailing_sl,
                trail_rupees=cfg.trail_rupees
            )
            if trade:
                all_trades.append(trade)

    trades = pd.DataFrame(all_trades)
    if trades.empty:
        print("No trades generated.")
        return trades

    trades.sort_values(by=["date", "ticker", "entry_time"], inplace=True)
    trades.to_csv(cfg.out_file, index=False)
    return trades

# =========================
# METRICS
# =========================
def max_drawdown(series: pd.Series) -> float:
    cum = series.cumsum()
    peak = cum.cummax()
    dd = cum - peak
    return float(dd.min())

def summarize(trades: pd.DataFrame):
    print("\n=== OVERALL METRICS (NET) ===")
    n = len(trades)
    wins = (trades["net_pnl"] > 0).sum()
    win_rate = 100.0 * wins / n if n else 0.0
    gross = trades["gross_pnl"].sum()
    charges = trades["charges"].sum()
    net = trades["net_pnl"].sum()
    mdd = max_drawdown(trades["net_pnl"])

    print(f"Trades: {n} | Win rate: {win_rate:.1f}%")
    print(f"Gross P&L: ₹{gross:,.2f} | Charges: ₹{charges:,.2f} | Net P&L: ₹{net:,.2f}")
    print(f"Max Drawdown (net): ₹{mdd:,.2f}")

    print("\n=== BY TICKER ===")
    by_t = trades.groupby("ticker").agg(
        n=("net_pnl","count"),
        wins=("net_pnl", lambda x: (x>0).sum()),
        gross=("gross_pnl","sum"),
        charges=("charges","sum"),
        net=("net_pnl","sum"),
        win_rate=("net_pnl", lambda x: 100.0*(x>0).mean())
    ).reset_index()
    by_t["win_rate"] = by_t["win_rate"].round(1)
    print(by_t.to_string(index=False))

    print("\n=== BY DIRECTION ===")
    by_dir = trades.groupby("direction").agg(
        n=("net_pnl","count"),
        wins=("net_pnl", lambda x: (x>0).sum()),
        gross=("gross_pnl","sum"),
        charges=("charges","sum"),
        net=("net_pnl","sum"),
        win_rate=("net_pnl", lambda x: 100.0*(x>0).mean())
    ).reset_index()
    by_dir["win_rate"] = by_dir["win_rate"].round(1)
    print(by_dir.to_string(index=False))

    print("\nWrote trades to:", CFG.out_file)

# =========================
# MAIN
# =========================
if __name__ == "__main__":
    trades = run_backtest(CFG)
    if not trades.empty:
        summarize(trades)



1 Failed download:
['DUMMYTATAM.NS']: YFTzMissingError('possibly delisted; no timezone found')


  WARN: No data for DUMMYTATAM.NS

=== OVERALL METRICS (NET) ===
Trades: 168 | Win rate: 29.8%
Gross P&L: ₹15,519.64 | Charges: ₹37,661.80 | Net P&L: ₹-22,142.19
Max Drawdown (net): ₹-21,923.68

=== BY TICKER ===
       ticker  n  wins    gross  charges      net  win_rate
  ADANIENT.NS  4     1   832.85   896.37   -63.52      25.0
ADANIPORTS.NS  3     1   161.48   673.40  -511.91      33.3
APOLLOHOSP.NS  4     2   978.50   890.38    88.12      50.0
ASIANPAINT.NS  1     0   -29.21   224.28  -253.49       0.0
  AXISBANK.NS  1     1   225.42   224.60     0.82     100.0
BAJAJ-AUTO.NS  4     0  -922.00   892.70 -1814.70       0.0
BAJAJFINSV.NS  2     0  -108.60   448.42  -557.03       0.0
BAJFINANCE.NS  7     2  1295.13  1571.99  -276.87      28.6
       BEL.NS  3     2   932.23   673.90   258.34      66.7
BHARTIARTL.NS  5     2   638.99  1121.39  -482.41      40.0
     CIPLA.NS  3     2  1276.73   673.23   603.50      66.7
 COALINDIA.NS  3     0   -78.25   673.57  -751.83       0.0
   DRRE